In [4]:
import torch

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('./dataset/HR.csv')

In [7]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,part,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   part                   14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [9]:
data.part.unique()

array(['sales', 'accounting', 'hr', 'technical', 'support', 'management',
       'IT', 'product_mng', 'marketing', 'RandD'], dtype=object)

In [10]:
data.salary.unique()

array(['low', 'medium', 'high'], dtype=object)

In [11]:
data = data.join(pd.get_dummies(data.part)).join(pd.get_dummies(data.salary))

In [12]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,part,salary,...,hr,management,marketing,product_mng,sales,support,technical,high,low,medium
0,0.38,0.53,2,157,3,0,1,0,sales,low,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,sales,medium,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,sales,medium,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,sales,low,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,sales,low,...,0,0,0,0,1,0,0,0,1,0


In [13]:
data.drop(columns=['part', 'salary'], inplace=True)

In [14]:
data.left.value_counts()

0    11428
1     3571
Name: left, dtype: int64

In [56]:
11428/(11428+3571)

0.7619174611640777

In [15]:
Y_data = data.left.values.reshape(-1, 1)

In [16]:
Y = torch.from_numpy(Y_data).type(torch.FloatTensor)

In [17]:
Y

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])

In [18]:
Y.shape

torch.Size([14999, 1])

In [19]:
[c for c in data.columns if c != 'left']

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'promotion_last_5years',
 'IT',
 'RandD',
 'accounting',
 'hr',
 'management',
 'marketing',
 'product_mng',
 'sales',
 'support',
 'technical',
 'high',
 'low',
 'medium']

In [20]:
X_data = data[[c for c in data.columns if c != 'left']].values

In [23]:
X = torch.from_numpy(X_data).type(torch.FloatTensor)

In [24]:
X.shape

torch.Size([14999, 20])

# 创建模型

In [25]:
from torch import nn

In [26]:
class Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin_1 = nn.Linear(20, 64)
        self.lin_2 = nn.Linear(64, 64)
        self.lin_3 = nn.Linear(64, 1)
        self.activate = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.lin_1(input)
        x = self.activate(x)
        x = self.lin_2(x)
        x = self.activate(x)
        x = self.lin_3(x)
        x = self.sigmoid(x)
        return x

### 我们将定义一个小函数来创建我们的模型和优化器，以便将来可以重用。

In [27]:
lr = 0.0001

In [28]:
def get_model():
    model = Logistic()
    return model, torch.optim.Adam(model.parameters(), lr=lr)

## 定义损失函数

In [29]:
loss_fn = nn.BCELoss()

In [30]:
model, opt = get_model()

In [31]:
len(data)

14999

In [32]:
batch = 64
no_of_batches = len(data)//batch
epochs = 100

In [33]:
for epoch in range(epochs):
    for i in range(no_of_batches):
        start = i*batch
        end = start + batch
        x = X[start: end]
        y = Y[start: end]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
print('epoch:', epoch, '   ', 'loss:', loss_fn(model(X), Y))

epoch: 99     loss: tensor(0.5176, grad_fn=<BinaryCrossEntropyBackward>)


# 使用dataset进行重构

PyTorch有一个抽象的Dataset类。Dataset可以是任何具有__len__函数和__getitem__作为对其进行索引的方法的函数。 本教程将通过示例将自定义HRDataset类创建为的Dataset的子类。

PyTorch的TensorDataset 是一个包装张量的Dataset。通过定义索引的长度和方式，这也为我们提供了沿张量的第一维进行迭代，索引和切片的方法。这将使我们在训练的同一行中更容易访问自变量和因变量。

In [34]:
from torch.utils.data import TensorDataset

In [35]:
HRdataset = TensorDataset(X, Y)

In [36]:
model, opt = get_model()

In [37]:
for epoch in range(epochs):
    for i in range(no_of_batches):
        x, y = HRdataset[i * batch: i * batch + batch]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
print('epoch:', epoch, '   ', 'loss:', loss_fn(model(X), Y))

epoch: 99     loss: tensor(0.5395, grad_fn=<BinaryCrossEntropyBackward>)


# 使用DataLoader进行重构

Pytorch DataLoader负责管理批次。

DataLoader从Dataset创建。

DataLoader使遍历批次变得更容易。DataLoader会自动为我们提供每个小批量。

无需使用 HRdataset[i * batch: i * batch + batch]

In [38]:
from torch.utils.data import DataLoader

HR_ds = TensorDataset(X, Y)
HR_dl = DataLoader(HR_ds, batch_size=batch)

In [39]:
model, opt = get_model()

In [40]:
for epoch in range(epochs):
    for x, y in HR_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
print('epoch:', epoch, '   ', 'loss:', loss_fn(model(X), Y))

epoch: 99     loss: tensor(0.5204, grad_fn=<BinaryCrossEntropyBackward>)


# 添加验证

前面我们只是试图建立一个合理的训练循环以用于我们的训练数据。实际上，您始终还应该具有一个验证集，以识别您是否过度拟合。

训练数据的乱序（shuffle）对于防止批次与过度拟合之间的相关性很重要。另一方面，无论我们是否乱序验证集，验证损失都是相同的。由于shufle需要额外的开销，因此shuffle验证数据没有任何意义。

我们将为验证集使用批大小，该批大小是训练集的两倍。这是因为验证集不需要反向传播，因此占用的内存更少（不需要存储梯度）。我们利用这一优势来使用更大的批量，并更快地计算损失。

!pip install sklearn -i https://pypi.douban.com/simple/

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)

In [44]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((11249, 20), (11249, 1), (3750, 20), (3750, 1))

In [45]:
train_x = torch.from_numpy(train_x).type(torch.FloatTensor)
test_x = torch.from_numpy(test_x).type(torch.FloatTensor)
train_y = torch.from_numpy(train_y).type(torch.FloatTensor)
test_y = torch.from_numpy(test_y).type(torch.FloatTensor)

In [46]:
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)

valid_ds = TensorDataset(test_x, test_y)
valid_dl = DataLoader(valid_ds, batch_size=batch * 2)

# 定义计算正确率函数

In [49]:
def accuracy(out, yb):
    preds = (out>0.5).type(torch.IntTensor)
    return (preds == yb).float().mean()

model.train()在训练之前调用代表训练模式

model.eval() 推理之前进行调用代表推理模式

不同的模式仅会在使用nn.BatchNorm2d ，nn.Dropout等层时以确保这些不同阶段的行为正确。

In [54]:
epochs = 500

In [55]:
model, opt = get_model()

for epoch in range(epochs+1):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_fn(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()
    if epoch%50==0:
        model.eval()
        with torch.no_grad():
            valid_loss = sum(loss_fn(model(xb), yb) for xb, yb in valid_dl)
            acc_mean = np.mean([accuracy(model(xb), yb) for xb, yb in valid_dl])
        print(epoch, valid_loss / len(valid_dl), acc_mean)

0 tensor(0.5753) 0.7549068
50 tensor(0.3225) 0.8585389
100 tensor(0.2648) 0.898835
150 tensor(0.2400) 0.91091007
200 tensor(0.2246) 0.9196683
250 tensor(0.2125) 0.92663103
300 tensor(0.2198) 0.92376643
350 tensor(0.2107) 0.92506856
400 tensor(0.2276) 0.9196683
450 tensor(0.2129) 0.923506
500 tensor(0.2130) 0.92663103


# 优化

In [57]:
class Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin_1 = nn.Linear(20, 64)
        self.lin_2 = nn.Linear(64, 64)
        self.lin_3 = nn.Linear(64, 64)
        self.lin_4 = nn.Linear(64, 1)
        self.activate = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.lin_1(input)
        x = self.activate(x)
        x = self.lin_2(x)
        x = self.activate(x)
        x = self.lin_3(x)
        x = self.activate(x)
        x = self.lin_4(x)
        x = self.sigmoid(x)
        return x

In [67]:
model, opt = get_model()

acc_val = []
acc_train = []

for epoch in range(epochs+1):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_fn(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()
    if epoch%50==0:
        model.eval()
        with torch.no_grad():
            valid_loss = sum(loss_fn(model(xb), yb) for xb, yb in valid_dl)
            acc_mean_train = np.mean([accuracy(model(xb), yb) for xb, yb in train_dl])
            acc_mean_val = np.mean([accuracy(model(xb), yb) for xb, yb in valid_dl])
        acc_train.append(acc_mean_train)
        acc_val.append(acc_mean_val)
        print(epoch, valid_loss / len(valid_dl), acc_mean_train, acc_mean_val)

0 tensor(0.5742) 0.76320267 0.7549068
50 tensor(0.2785) 0.88764817 0.89180374
100 tensor(0.2118) 0.9235763 0.9269874
150 tensor(0.1965) 0.92992663 0.9289748
200 tensor(0.1926) 0.93254286 0.93236023
250 tensor(0.1852) 0.93928283 0.9362665
300 tensor(0.1778) 0.9432779 0.93965185
350 tensor(0.1758) 0.94339377 0.93939143
400 tensor(0.2052) 0.9267651 0.91820174
450 tensor(0.1701) 0.94368726 0.939131
500 tensor(0.1655) 0.9481607 0.93955594


# 创建fit（）和get_data（）

In [72]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [73]:
import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

In [74]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

### 现在，我们获取数据加载器和拟合模型的整个过程可以在3行代码中运行：

In [77]:
train_dl, valid_dl = get_data(train_ds, valid_ds, batch)
model, opt = get_model()
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

0 0.5466923921585083
1 0.5437768836975098
2 0.5458150820096334
3 0.5379680653254191
4 0.5520771965344747
5 0.5333976115544637
6 0.5301197660128275
7 0.5239578514099121
8 0.5194751345634461
9 0.5135513541857402
10 0.5126034033457438
11 0.5128153882344564
12 0.49411540430386863
13 0.4861029712994893
14 0.48530904811223347
15 0.47241837673187254
16 0.48379859307607015
17 0.45810359698931374
18 0.451055920346578
19 0.44228789970080057
20 0.43615612886746724
21 0.42766183932622276
22 0.4213903223514557
23 0.4172084908803304
24 0.4100531432946523
25 0.403511372423172
26 0.3971054848988851
27 0.3925201852162679
28 0.3868228485584259
29 0.3867359171072642
30 0.37675094564755757
31 0.3722356454372406
32 0.3681845734278361
33 0.3643391172091166
34 0.375385142993927
35 0.3803580154577891
36 0.3540113541762034
37 0.3471658375898997
38 0.3483211691220601
39 0.3409267953713735
40 0.3389663805802663
41 0.3331018171946208
42 0.33961131242116294
43 0.32884247654279075
44 0.3257884572505951
45 0.3270215